In [1]:
### Libraries
import pyodbc 
import time
import pandas as pd
from IPython.display import display

### Connection parameters
dsn = 'IRIS IntegeratedML monitor'
server = 'irisimlsvr'
port = '51773'
database = 'USER' 
username = 'SUPERUSER' 
password = 'SYS' 
cnxn = pyodbc.connect('DRIVER={InterSystems ODBC35};SERVER='+server+';PORT='+port+';DATABASE='+database+';UID='+username+';PWD='+ password)

### charset stuffs...
cnxn.setdecoding(pyodbc.SQL_CHAR, encoding='utf8')
cnxn.setdecoding(pyodbc.SQL_WCHAR, encoding='utf8')
cnxn.setencoding(encoding='utf8')

### Model's parameters
dataTable = 'MedicalAppointments'
dataColumn = 'Show'
modelName = 'AppointmentsShowPredection'
trainLen = 500

### Get a cursor
cursor = cnxn.cursor()

In [2]:
### Clean previous runs
df1 = pd.read_sql(
    "SELECT COUNT(*) AS ModelExists FROM INFORMATION_SCHEMA.ML_TRAINED_MODELS WHERE MODEL_NAME = '%s'"
    % (modelName), 
    cnxn
)
if (df1['ModelExists'][0] == 1):
    print('Deleting previous model...')
    cursor.execute("DROP MODEL %s" % (modelName))
    print('Model deleted')

Deleting previous model...
Model deleted


In [3]:
### Create a model for predict patient's appointments misses
### Note: I used seed parameter to ensure reproducibility
cursor.execute(
    "CREATE MODEL %s PREDICTING (%s) FROM %s USING {\"seed\": 3}" 
    % (modelName, dataColumn, dataTable)
)
print('Training model (this could take a while)...')
cursor.execute(
    "TRAIN MODEL %s FROM %s WHERE ID <= %s USING {\"seed\": 3}" 
    % (modelName, dataTable, trainLen)
)
print('Model trained')
cnxn.commit()

Training model (this could take a while)...
Model trained


In [4]:
### Display model information
cursor.execute("SELECT * FROM INFORMATION_SCHEMA.ML_TRAINED_MODELS")
df1 = pd.read_sql("SELECT * FROM INFORMATION_SCHEMA.ML_TRAINED_MODELS", cnxn)
display(df1)

,MODEL_NAME,TRAINED_MODEL_NAME,PROVIDER,TRAINED_TIMESTAMP,MODEL_TYPE,MODEL_INFO
0,AppointmentsShowPredection,AppointmentsShowPredection2,AutoML,2020-07-09 04:22:54.128,classification,"ModelType:Logistic Regression, Package:sklearn..."
1,bc,bc2,AutoML,2020-07-08 11:25:26.578,classification,"ModelType:Logistic Regression, Package:sklearn..."
2,SurviveClassification,SurviveClassification2,AutoML,2020-07-08 11:27:08.419,classification,"ModelType:Random Forest, Package:sklearn, Prob..."
3,CampaignModel,CampaignModel2,AutoML,2020-07-08 11:56:59.095,classification,"ModelType:Logistic Regression, Package:sklearn..."


In [5]:
### Display prediction against real patient's appointments presence
df1 = pd.read_sql(
    "SELECT PREDICT(%s) As Predicted, Show FROM %s WHERE ID <= %s" 
    % (modelName, dataTable, trainLen), 
    cnxn
)
display(df1)

,Predicted,Show
0,0,False
1,0,False
2,0,False
3,0,False
4,0,False
...,...,...
495,1,True
496,0,True
497,1,True
498,1,True


In [6]:
### Model validation
cursor.execute(
    "VALIDATE MODEL %s FROM %s WHERE ID < %s USING {\"seed\": 3}" 
    % (modelName, dataTable, 1000)
)
df5 = pd.read_sql(
    "SELECT * FROM INFORMATION_SCHEMA.ML_VALIDATION_METRICS WHERE MODEL_NAME = '%s'" 
    % (modelName), cnxn
)
df6 = df5.pivot(index='VALIDATION_RUN_NAME', columns='METRIC_NAME', values='METRIC_VALUE')
display(df6)

METRIC_NAME,Accuracy,F-Measure,Precision,Recall
VALIDATION_RUN_NAME,,,,
AppointmentsShowPredection21,0.83,0.89,0.95,0.83


As we can see, model's accuracy is about 83%.  
MyMetric.IntegratedMLModelsValidation application monitor class let you to access such metrics. Thus it's possible to IRIS' ^%SYSMONMGR monitor utility being aware about your ML models perfomance.

In [7]:
!pip install /tmp/irisnative-1.2.0-cp34-abi3-linux_x86_64.whl

In [8]:
### IRIS Python Native API
import irisnative

### Native API connection's parameters
ip = "irisimlsvr"
port = 51773
namespace = "USER"
username = "SUPERUSER"
password = "SYS"

### Create database connection and IRIS instance
nativeConn = irisnative.createConnection(ip, port, namespace, username, password)
dbnative = irisnative.createIris(nativeConn)

### Get the last validation parameter for each IntegeratedML model
print(dbnative.classMethodValue("MyMetric.Install", "Test", "MyMetric.IntegratedMLModelsValidation"))

MyMetric - IntegratedMLModelsValidation   AppointmentsShowPredection26556915778926448   Serial value of object or the object identity:UMyMetric||IntegratedMLModelsValidation||AppointmentsShowPredection26556915778926448.MyMetric.Sample.IntegratedMLModelsValidation
   Accuracy:                   .83
   F-Measure:                  .89
   Precision:                  .95
   Recall:                     .83
   Name of the model definition:APPOINTMENTSSHOWPREDECTION
   Name of the trained model being validated:AppointmentsShowPredection26556915778926448
   Validation error (if encountered):1




Same results also could see by CSP interface: http://localhost:8092/csp/user/MyMetric.Sample.IntegratedMLModelsValidation.cls

With such functionality, someone could setup an alert into ^%SYSMONMGR monitor utility, checking model's performance metrics against thresholds. This way, IntegeratedML applications performance issues could be quickly notified.

In [9]:
### Close the database and native API connections
nativeConn.close()
cnxn.close()